In [2]:
import cv2
import numpy as np
import pandas as pd
from datetime import datetime
from tensorflow.keras.models import load_model

# Loading  pre-trained models
age_net = cv2.dnn.readNetFromCaffe(
    "/kaggle/input/age-and-emotion-setup/deploy_age.prototxt",  # Download the required prototxt and model files
    "/kaggle/input/age-and-emotion-setup/age_net.caffemodel"
)
emotion_model = load_model("/kaggle/input/age-and-emotion-setup/model_a1.json")  # Replace with your custom emotion model

# Age categories
AGE_BUCKETS = ["(0-2)", "(4-6)", "(8-12)", "(13-19)", "(20-29)", "(30-39)", "(40-50)", "(60+)"]

# Emotion categories
EMOTION_LABELS = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

# Initializimng CSV file for logging
csv_file = "movie_theatre_log.csv"
df = pd.DataFrame(columns=["Age", "Emotion", "Entry Time"])
df.to_csv(csv_file, index=False)

# Processing frame for age detection
def detect_age_and_emotion(frame, face):
    # Extract face region
    (x, y, w, h) = face
    face_roi = frame[y:y+h, x:x+w]
    blob = cv2.dnn.blobFromImage(face_roi, 1.0, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)

    # Predicting age
    age_net.setInput(blob)
    age_preds = age_net.forward()
    age_index = age_preds[0].argmax()
    age_label = AGE_BUCKETS[age_index]

    # Predicting emotion if age is between 13 and 60
    emotion_label = None
    if "(13-19)" in age_label or "(20-29)" in age_label or "(30-39)" in age_label or "(40-50)" in age_label:
        gray_face = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
        gray_face = cv2.resize(gray_face, (48, 48)) / 255.0
        gray_face = np.expand_dims(gray_face, axis=0)
        gray_face = np.expand_dims(gray_face, axis=-1)
        emotion_preds = emotion_model.predict(gray_face)
        emotion_label = EMOTION_LABELS[np.argmax(emotion_preds)]

    return age_label, emotion_label

def log_data(age, emotion):
    entry_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    data = {"Age": age, "Emotion": emotion, "Entry Time": entry_time}
    df = pd.DataFrame([data])
    df.to_csv(csv_file, mode="a", header=False, index=False)

# Initializing webcam
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Converting to grayscale and detect faces
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))

    for face in faces:
        (x, y, w, h) = face
        age_label, emotion_label = detect_age_and_emotion(frame, face)

        # Determining the message and rectangle color
        if "(0-2)" in age_label or "(4-6)" in age_label or "(8-12)" in age_label or "(60+)" in age_label:
            message = "Not Allowed"
            color = (0, 0, 255)  # Red
        else:
            message = f"Emotion: {emotion_label}"
            color = (0, 255, 0)  # Green

        # Drawing rectangle and display message
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, f"Age: {age_label}", (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        cv2.putText(frame, message, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # Log data
        log_data(age_label, emotion_label if emotion_label else "N/A")

    # Display the video feed
    cv2.imshow("Age and Emotion Detection", frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


ValueError: File format not supported: filepath=/kaggle/input/age-and-emotion-setup/model_a1.json. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(/kaggle/input/age-and-emotion-setup/model_a1.json, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).